1. 安装组件

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install --upgrade transformers trl peft bitsandbytes accelerate datasets pandas modelscope swanlab

2. 加载数据

In [3]:
from modelscope.msdatasets import MsDataset
import json
import random
import os

# 设置随机种子
random.seed(42)

# 加载数据集
ds = MsDataset.load(
    'krisfu/delicate_medical_r1_data',
    subset_name='default',
    split='train',
    trust_remote_code=True  # 显式声明
)
data_list = list(ds)
random.shuffle(data_list)

# 划分训练集和验证集
split_idx = int(len(data_list) * 0.9)
train_data = data_list[:split_idx]
val_data = data_list[split_idx:]

# 创建 data 目录（如果不存在）
os.makedirs('data', exist_ok=True)

# 保存到 data 目录下的 jsonl 文件
with open('data/train.jsonl', 'w', encoding='utf-8') as f:
    for item in train_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

with open('data/val.jsonl', 'w', encoding='utf-8') as f:
    for item in val_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"The dataset has been split successfully.")
print(f"Train Set Size: {len(train_data)}")
print(f"Val Set Size: {len(val_data)}")

2025-08-27 12:26:55,287 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from delicate_medical_r1_data. Please make sure that you can trust the external codes.
2025-08-27 12:26:55,943 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from krisfu/delicate_medical_r1_data. Please make sure that you can trust the external codes.
2025-08-27 12:26:55,944 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from krisfu/delicate_medical_r1_data. Please make sure that you can trust the external codes.
2025-08-27 12:26:55,944 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from krisfu/delicate_medical_r1_data. Please make sure that you can trust the external codes.


The dataset has been split successfully.
Train Set Size: 2166
Val Set Size: 241


3. 开始训练

In [5]:
# -*- coding: utf-8 -*-
"""
医学问答模型QLoRA高效微调脚本 - 【V8.0 极致注释修复版】

【核心修复】
1.  [修复] 彻底解决 `UnboundLocalError`: 将所有import语句移至文件顶部，遵循Python最佳实践，解决了局部作用域导致的变量未定义问题。
2.  [修复] 修正模型保存逻辑: 在最后一步确保保存的是合并后的完整模型(`merged_model`)，而不是训练时使用的量化模型(`model`)。
3.  [新增] “官方改装版”模型导出: 在训练结束后，自动将LoRA权重与基础模型合并，生成一个独立的、开箱即用的完整模型，极大方便了后续的评估和部署。

【保留功能】
1.  QLoRA微调、Flash Attention 2、完整的训练流程。
"""

# =====================================================================================
# 【！！！关键！！！】环境变量配置 - 必须在所有import之前
# =====================================================================================
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["DISABLE_IPEX"] = "1"
os.environ["DEEPSPEED_XPU_ENABLE"] = "0"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# =====================================================================================
# 导入所有必需的库 (所有import都应在此处)
# =====================================================================================
import json
import pandas as pd
import torch
from datasets import Dataset
from modelscope import snapshot_download
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, # <--- 从transformers库导入核心的模型类
    TrainingArguments, 
    Trainer, 
    DataCollatorForSeq2Seq, 
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel # <--- 从peft库导入LoRA相关工具

# =====================================================================================
# 【！！！核心重点！！！】全局配置与超参数
# =====================================================================================

# --- 路径配置 (Path Configuration) ---
# 脚本会按照这些路径创建文件夹、读取数据和保存模型。
BASE_DIR = "."
DATA_DIR = os.path.join(BASE_DIR, "data")
FORMATTED_DATA_DIR = os.path.join(BASE_DIR, "data_formatted")
MODEL_CACHE_DIR = os.path.join(BASE_DIR, "model_cache")
OUTPUT_DIR = os.path.join(BASE_DIR, "output")
CHECKPOINTS_DIR = os.path.join(OUTPUT_DIR, "checkpoints") # 训练过程中的检查点（临时存档）
FINAL_ADAPTER_DIR = os.path.join(OUTPUT_DIR, "final_model") # 【注意】现在保存的是完整模型，文件夹名已修改

# --- 模型配置 (Model Configuration) ---
MODEL_ID = "qwen/Qwen3-1.7B" # 你选择的基础模型ID，脚本会自动从网上下载

# 【系统指令 SYSTEM_PROMPT】
#   - 是什么：给模型设定的“人设”或“行为准则”，告诉它应该扮演什么角色。
#   - 【！！！核心重点！！！】这里的PROMPT必须与你评估脚本中使用的SYSTEM_PROMPT完全一致！
#     这相当于训练和考核时使用同一份说明书，否则模型会感到困惑，导致评估结果不准确。
SYSTEM_PROMPT = """你是一个专业、严谨的AI医学助手。你的任务是根据用户提出的问题，提供准确、易懂且具有安全提示的健康信息。请记住，你的回答不能替代执业医师的诊断，必须在回答结尾处声明这一点。"""

# 【最大长度 MAX_LENGTH】
#   - 是什么：一条训练数据（问题+答案）被转换成数字后，允许的最大长度。
#   - 为什么是2048：对于Qwen3-1.7B模型来说，这是一个比较均衡的值，能容纳大部分的问答对。
#   - 改了会怎样：如果你的问答文本非常长，需要适当增加这个值（如4096），但这会显著增加显存消耗。如果设置太小，过长的文本会被截断，导致信息丢失。
MAX_LENGTH = 2048

# --- 【！！！核心重点！！！】训练超参数 (Hyperparameters) ---
# 这些参数直接影响模型的学习效果、速度和资源消耗，是微调的“灵魂”。

# 【学习率 Learning Rate】
#   - 是什么：可以理解为模型学习时“每一步走多大”。
#   - 为什么是1e-4：对于LoRA这种只训练一小部分参数的方法，可以设置比全量微调（通常是2e-5）稍大的学习率，让“适配器”参数学得更快。1e-4是QLoRA一个经过大量验证的、效果很好的起始值。
#   - 改了会怎样：太高（如1e-3）可能导致模型“跑偏”，训练不稳定；太低（如1e-5）学习会非常慢，甚至“学不动”。
LEARNING_RATE = 1e-4

# 【单设备批量大小 PER_DEVICE_TRAIN_BATCH_SIZE】
#   - 是什么：一次性喂给GPU多少条数据进行训练。
#   - 为什么是2：这是为了在有限的显存（如24GB）下运行而做的优化。值越小，单次计算的显存占用越低。
#   - 改了会怎样：值越大，训练速度越快，但显存占用越高。如果设置过大，会导致“CUDA out of memory”错误。
PER_DEVICE_TRAIN_BATCH_SIZE = 1

# 【梯度累积步数 GRADIENT_ACCUMULATION_STEPS】
#   - 是什么：通过“攒”梯度的方式，在不增加显存的情况下，实现等效于更大Batch Size的训练效果。
#   - 为什么是16：配合上面的`BATCH_SIZE = 2`，我们的“有效批量大小” (Effective Batch Size) = 2 * 16 = 32。这通常是一个能让模型稳定学习的批量大小。
#   - 改了会怎样：它和`BATCH_SIZE`是跷跷板关系。显存不足时，降低`BATCH_SIZE`，同时提高此值，以保持“有效批量大小”不变。
GRADIENT_ACCUMULATION_STEPS = 32

# 【训练轮数 NUM_TRAIN_EPOCHS】
#   - 是什么：将整个训练数据集从头到尾完整地学习多少遍。
#   - 为什么是2：对于微调任务，通常1-3个epoch就足够了，可以快速看到效果。
#   - 改了会怎样：太少可能导致模型“没学会”；太多则可能发生“过拟合”（模型只会死记硬背训练数据，丧失了泛化能力）。
NUM_TRAIN_EPOCHS = 2

EVAL_STEPS = 20
SAVE_STEPS = 400
LOGGING_STEPS = 10

# --- 【！！！核心重点！！！】LoRA配置 (LoRA Configuration) ---

# 【LoRA秩 R】
#   - 是什么：LoRA“升级套件”的“复杂度”或“大小”。
#   - 为什么是16：R值通常在8, 16, 32, 64中选择。16是一个很好的起始点，在性能和参数量之间取得了平衡。
#   - 改了会怎样：R值越大，可训练的参数越多，理论上拟合能力越强，但显存占用也越大，且过大也可能导致过拟合。
LORA_R = 16

# 【LoRA缩放因子 Alpha】
#   - 是什么：一个缩放参数，用来调整LoRA权重的幅度。
#   - 为什么是32：通常建议设置为R的两倍（`alpha = 2 * r`），这是一个经验性的、效果很好的设置。
#   - 改了会怎样：它与学习率有类似的作用，调整它会影响LoRA模块的权重大小。保持`2*r`的比例通常是最佳实践。
LORA_ALPHA = 32

LORA_DROPOUT = 0.05 # 防止过拟合的技术，训练时随机“丢弃”一些神经元。

# 【目标模块 TARGET_MODULES】
#   - 是什么：告诉LoRA应该在模型的哪些部分“加装升级套件”。
#   - 为什么是这几个：`q_proj`, `k_proj`, `v_proj`, `o_proj`是Transformer模型中注意力机制的核心组件，对它们进行微调通常效果最显著。
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]

# --- 运行配置 ---
EFFECTIVE_BATCH_SIZE = PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS
RUN_NAME = f"qwen-medical-qlora-lr{LEARNING_RATE}-bs{EFFECTIVE_BATCH_SIZE}-r{LORA_R}"

# =====================================================================================
# 工具函数 (无需修改)
# =====================================================================================

def check_environment():
    # ... (此函数无需修改)
    if not torch.cuda.is_available(): raise RuntimeError("❌ CUDA不可用！")
    torch.cuda.empty_cache()
    print("✅ CUDA环境检查通过！")

def setup_directories():
    # ... (此函数无需修改)
    for directory in [DATA_DIR, FORMATTED_DATA_DIR, MODEL_CACHE_DIR, CHECKPOINTS_DIR, FINAL_ADAPTER_DIR]:
        os.makedirs(directory, exist_ok=True)
    print("✅ 所有项目目录已准备就绪")

def dataset_jsonl_transfer(origin_path, new_path):
    # ... (此函数无需修改)
    messages = []
    with open(origin_path, "r", encoding="utf-8") as file:
        for line in file:
            try:
                data = json.loads(line.strip())
                messages.append({
                    "input": data["question"],
                    "output": f'<|FunctionCallBegin|>{data["think"]}<|FunctionCallEnd|>\n{data["answer"]}'
                })
            except (KeyError, json.JSONDecodeError): continue
    with open(new_path, "w", encoding="utf-8") as file:
        for message in messages:
            file.write(json.dumps(message, ensure_ascii=False) + "\n")
    print(f"✅ 数据转换完成: {len(messages)} 条有效数据")

def process_func(example, tokenizer):
    # ... (此函数无需修改)
    instruction = tokenizer(f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)
    response = tokenizer(example['output'], add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": [1] * len(input_ids), "labels": labels}

# =====================================================================================
# 【！！！核心重点！！！】主训练流程
# =====================================================================================

def main():
    """主训练流程"""
    print("🚀 开始QLoRA医学问答微调流程...")
    
    # --- 步骤1: 环境与路径初始化 ---
    check_environment()
    setup_directories()

    # --- 步骤2: 加载分词器并扩展 ---
    print("\n📥 正在加载基础模型的分词器...")
    model_dir = snapshot_download(MODEL_ID, cache_dir=MODEL_CACHE_DIR, revision="master")
    tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False, trust_remote_code=True)
    
    # 【关键】添加新的特殊词汇，让模型能“听懂”我们的特殊指令
    special_tokens_dict = {'additional_special_tokens': ['<|FunctionCallBegin|>', '<|FunctionCallEnd|>']}
    tokenizer.add_special_tokens(special_tokens_dict)
    if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
    print("✅ 分词器加载并扩展完成")

    # --- 步骤3: 配置QLoRA量化 ---
    # 这是实现“低显存”微调的核心。它将模型参数从32位浮点数压缩到4位整数，大大减少显存占用。
    print("\n⚙️ 配置4-bit量化参数(QLoRA)...")
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_use_double_quant=True,
    )

    # --- 步骤4: 加载量化模型 ---
    print("\n🔧 加载量化后的基础模型...")
    torch.cuda.empty_cache()
    
    # 优先尝试使用Flash Attention 2，这是一种高效的注意力计算方法，可以提速并节省显存。
    # 如果环境不支持，会自动切换到标准的注意力机制。
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_dir, device_map="auto", torch_dtype=torch.bfloat16,
            quantization_config=quantization_config,
            attn_implementation="flash_attention_2",
        )
        print("✅ 量化模型加载完成 (Flash Attention 2 已启用)")
    except Exception as e:
        print(f"⚠️ Flash Attention 2不可用，将使用标准注意力机制: {e}")
        model = AutoModelForCausalLM.from_pretrained(
            model_dir, device_map="auto", torch_dtype=torch.bfloat16,
            quantization_config=quantization_config,
        )
        print("✅ 量化模型加载完成 (标准注意力机制)")
    
    # 【关键】调整模型词嵌入层的大小，以匹配我们扩展后的分词器。这是避免后续尺寸不匹配错误的核心步骤。
    model.resize_token_embeddings(len(tokenizer))
    model = prepare_model_for_kbit_training(model)

    # --- 步骤5: 应用LoRA适配器 ---
    lora_config = LoraConfig(
        r=LORA_R, lora_alpha=LORA_ALPHA, target_modules=TARGET_MODULES,
        lora_dropout=LORA_DROPOUT, bias="none", task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters() # 打印出可训练参数的比例，你会发现它非常小！
    print("✅ LoRA适配器已成功应用")

    # --- 步骤6 & 7: 数据准备与预处理 ---
    # (这部分代码无需修改，它会自动处理数据格式转换和Tokenization)
    print("\n📊 准备并预处理数据...")
    train_original_path = os.path.join(DATA_DIR, "train.jsonl")
    val_original_path = os.path.join(DATA_DIR, "val.jsonl")
    train_formatted_path = os.path.join(FORMATTED_DATA_DIR, "train_formatted.jsonl")
    val_formatted_path = os.path.join(FORMATTED_DATA_DIR, "val_formatted.jsonl")
    if not os.path.exists(train_formatted_path): dataset_jsonl_transfer(train_original_path, train_formatted_path)
    if not os.path.exists(val_formatted_path): dataset_jsonl_transfer(val_original_path, val_formatted_path)
    train_dataset = Dataset.from_pandas(pd.read_json(train_formatted_path, lines=True))
    eval_dataset = Dataset.from_pandas(pd.read_json(val_formatted_path, lines=True))
    tokenized_train_dataset = train_dataset.map(lambda x: process_func(x, tokenizer), remove_columns=train_dataset.column_names)
    tokenized_eval_dataset = eval_dataset.map(lambda x: process_func(x, tokenizer), remove_columns=eval_dataset.column_names)
    print("✅ 数据准备与预处理完成")

    # --- 步骤8 & 9: 配置训练参数并初始化Trainer ---
    print("\n⚙️ 配置训练参数并初始化训练器...")
    training_args = TrainingArguments(
        output_dir=os.path.join(CHECKPOINTS_DIR, RUN_NAME),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        learning_rate=LEARNING_RATE,
        num_train_epochs=NUM_TRAIN_EPOCHS,
        lr_scheduler_type="cosine",
        warmup_ratio=0.1,
        logging_steps=LOGGING_STEPS,
        eval_strategy="steps",
        eval_steps=EVAL_STEPS,
        save_steps=SAVE_STEPS,
        save_total_limit=2,
        bf16=True, # 使用bf16混合精度训练，可以提速并节省显存
        gradient_checkpointing=True, # 关键的显存优化技术，用时间换空间
        optim="paged_adamw_8bit", # 使用分页优化器，进一步节省显存
        remove_unused_columns=False,
        #report_to="none", # 禁用外部日志上报
    )
    trainer = Trainer(
        model=model, args=training_args, train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_eval_dataset,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
    )
    print("✅ 训练器初始化完成")

    # --- 步骤10: 开始训练 ---
    print("\n🚀 开始模型训练...")
    trainer.train()
    print("🎉 训练完成!")

    # --- 【！！！核心重点：合并权重并保存为完整模型！！！】 ---
    print("\n🔗 正在将LoRA适配器权重合并到基础模型中，以生成可独立部署的完整模型...")
    
    # 释放显存，为加载全精度模型做准备
    del model
    del trainer
    torch.cuda.empty_cache()
    
    # 重新加载全精度的基础模型
    base_model = AutoModelForCausalLM.from_pretrained(
        model_dir, torch_dtype=torch.bfloat16, device_map="auto", trust_remote_code=True
    )
    # 再次扩展词汇表，确保结构一致
    base_model.resize_token_embeddings(len(tokenizer))

    # 从最新的检查点加载我们训练好的LoRA适配器权重
    lora_model_path = os.path.join(CHECKPOINTS_DIR, RUN_NAME)
    latest_checkpoint = max([d for d in os.listdir(lora_model_path) if d.startswith("checkpoint-")], key=lambda x: int(x.split("-")[-1]))
    adapter_path = os.path.join(lora_model_path, latest_checkpoint)
    print(f"    -> 正在从最新的检查点加载LoRA适配器: {adapter_path}")
    
    model_to_merge = PeftModel.from_pretrained(base_model, adapter_path)

    # 执行合并！这将返回一个全新的、完整的、微调后的模型
    merged_model = model_to_merge.merge_and_unload()
    print("✅ LoRA权重合并完成！")
    
    # --- 步骤11: 保存最终的完整模型 ---
    print("\n💾 正在保存合并后的完整模型...")
    final_model_path = os.path.join(FINAL_ADAPTER_DIR, RUN_NAME)
    
    # 【关键修复】确保我们保存的是`merged_model`，而不是旧的`model`
    merged_model.save_pretrained(final_model_path)
    tokenizer.save_pretrained(final_model_path)
    
    print(f"✅ 最终的“官方改装版”模型已保存到: {final_model_path}")
    print("\n🎉 QLoRA医学问答微调流程全部完成!")

# =====================================================================================
# 程序入口
# =====================================================================================
if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"\n❌ 程序执行失败: {e}")
        import traceback
        traceback.print_exc()
    finally:
        print("🔚 程序结束")

🚀 开始QLoRA医学问答微调流程...
✅ CUDA环境检查通过！
✅ 所有项目目录已准备就绪

📥 正在加载基础模型的分词器...


2025-08-27 14:45:14,158 - modelscope - INFO - Target directory already exists, skipping creation.


✅ 分词器加载并扩展完成

⚙️ 配置4-bit量化参数(QLoRA)...

🔧 加载量化后的基础模型...
⚠️ Flash Attention 2不可用，将使用标准注意力机制: /usr/local/lib/python3.11/site-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationESs


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 量化模型加载完成 (标准注意力机制)
trainable params: 6,422,528 || all params: 1,726,454,784 || trainable%: 0.3720
✅ LoRA适配器已成功应用

📊 准备并预处理数据...


...

🎉 训练完成!

🔗 正在将LoRA适配器权重合并到基础模型中，以生成可独立部署的完整模型...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

    -> 正在从最新的检查点加载LoRA适配器: ./output/checkpoints/qwen-medical-qlora-lr0.0001-bs32-r16/checkpoint-136


/usr/local/lib/python3.11/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)


✅ LoRA权重合并完成！

💾 正在保存合并后的完整模型...
✅ 最终的“官方改装版”模型已保存到: ./output/final_model/qwen-medical-qlora-lr0.0001-bs32-r16

🎉 QLoRA医学问答微调流程全部完成!
🔚 程序结束
